In [1]:
import numpy as np
import fitsio
import matplotlib.pyplot as plt
import os
import pandas as pd

from astropy.io import fits
from astropy.time import Time

import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)

In [2]:
def create_fits_TRAN(ID,files,classification):
    global MANTRA_NOT
    global MJD_NOT
    
    path = './data/'+classification+'/'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    filename = './data/{}/{}.fits'.format(classification,ID)
    
    if os.path.isfile(filename):
        return 3
    
    df = pd.read_table('../tao_data/transient_info.txt')
    CRTS_ID = np.array(df['#CRTS ID'])
    if sum(CRTS_ID == ID) != 0:
        RA = np.array(df[CRTS_ID == ID]['RA (J2000)'])[0]
        DEC = np.array(df[CRTS_ID == ID]['Dec (J2000)'])[0]
        UT_Date = np.array(df[CRTS_ID == ID]['UT Date'])[0]
        MAG = np.array(df[CRTS_ID == ID]['Mag'])[0]
        CSS_Images = np.array(df[CRTS_ID == ID]['CSS images'])[0]
        SDSS = np.array(df[CRTS_ID == ID]['SDSS'])[0]
        OTHERS = np.array(df[CRTS_ID == ID]['Others'])[0]
        FOLLOWED = np.array(df[CRTS_ID == ID]['Followed'])[0]
        LAST = np.array(df[CRTS_ID == ID]['Last'])[0]
        LC = np.array(df[CRTS_ID == ID]['LC'])[0]
        FC = np.array(df[CRTS_ID == ID]['FC'])[0]
        CLASS = np.array(df[CRTS_ID == ID]['Classification'])[0]

        N = len(files)
        SEQUENCE = np.zeros(N, dtype='S2')
        DATE = np.zeros(N, dtype='S7')
        FIELD_ID = np.zeros(N, dtype='S6')
        OBS_IN_SEQ =  np.zeros(N, dtype='S4')
        CUTOUT = np.zeros(N, dtype='S4')
        HDU_EXT = np.arange(2,N+2).astype('S4')
        MJD = np.zeros(N, dtype='S11')
        for i, file in enumerate(files):
            try:
                try:
                    hdul = fits.open('{}.fits'.format(file[:-12]))
                    MJD[i] = str(hdul[0].header['MJD'])
                except:
                    date_obs = str(hdul[0].header['DATE-OBS'])
                    time_obs = str(hdul[0].header['TIME-OBS'])
                    time = '{}T{}'.format(date_obs,time_obs)
                    MJD[i] = Time(time, format='isot').mjd
            except:
                MJD_NOT.append(file[:-12])
                return 2
            SEQUENCE[i] = file.split('/')[5].split('_')[0]
            DATE[i] = file.split('/')[5].split('_')[1]
            FIELD_ID[i] = file.split('/')[5].split('_')[2]
            OBS_IN_SEQ[i] = file.split('/')[5].split('_')[3]
            CUTOUT[i] = file.split('/')[5].split('_')[5]


        hdict = {'CRTS_ID': ID, 'RA_(J2000)': RA, 'Dec_(J2000)':DEC, 'N_Images': N, 
                'UT_Date': UT_Date, 'Mag': MAG, 'CSS_images': CSS_Images,
                 'SDSS': SDSS, 'Others': OTHERS, 'Followed': FOLLOWED, 'Last': LAST,
                 'LC': LC, 'FC': FC, 'Class': CLASS}
        data = {'HDU_Ext': HDU_EXT, 'Sequence': SEQUENCE, 'Date': DATE, 'MJD': MJD, 'Field_ID':FIELD_ID,
                'Obs_In_Seq': OBS_IN_SEQ, 'Cutout': CUTOUT}
        fitsio.write(filename, data, header=hdict)

        for file in files:
            img = np.load(file)
            fitsio.write(filename, img)
        return 0
    else:
        MANTRA_NOT.append(ID)
        return 1

In [4]:
MJD_NOT = []
MANTRA_NOT = []

classes = ['AGN','BZ','CV','SN','OTHER']
for classification in classes:
    print(classification)
    sucss = 0
    no_MANTRA = 0
    no_MJD = 0
    alr_ext = 0
    
    path = '../tao_data/Test/{}/'.format(classification)
    objects = !ls $path
    print('Objects in Test: {}'.format(len(objects)))
    for objct in objects:
        path_file = path+objct+'/*.npy'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_TRAN(ID,files,classification)
        if value == 0:
            sucss +=1
        elif value == 1:
            no_MANTRA += 1
        elif value == 2:
            no_MJD += 1
        else:
            alr_ext += 1
            
    path = '../tao_data/Complete/{}/'.format(classification)
    objects = !ls $path
    print('Objects in Complete: {}'.format(len(objects)))
    for objct in objects:
        path_file = path+objct+'/*.npy'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_TRAN(ID,files,classification)
        if value == 0:
            sucss +=1
        elif value == 1:
            no_MANTRA += 1
        elif value == 2:
            no_MJD += 1
        else:
            alr_ext += 1

    print('Successfull {} {}'.format(sucss,classification))
    print('no MANTRA {} {}'.format(no_MANTRA,classification))
    print('no MJD {} {}'.format(no_MJD,classification))
    print('Alr Exist {} {}'.format(alr_ext,classification))
    
    MANTRA_NOTDF = pd.DataFrame(MANTRA_NOT, columns=['ID'])
    MJD_NOTDF = pd.DataFrame(MJD_NOT, columns=['ID'])

MANTRA_NOTDF.to_pickle('../tao_data/TRAN_MANTRA_NOT.pkl')
MJD_NOTDF.to_pickle('../tao_data/TRAN_MJD_NOT.pkl')

AGN
Objects in Test: 28
Objects in Complete: 606
Successfull 604 AGN
no MANTRA 0 AGN
no MJD 20 AGN
Alr Exist 10 AGN
BZ
Objects in Test: 14
Objects in Complete: 239
Successfull 234 BZ
no MANTRA 0 BZ
no MJD 11 BZ
Alr Exist 8 BZ
CV
Objects in Test: 54
Objects in Complete: 776
Successfull 765 CV
no MANTRA 4 CV
no MJD 7 CV
Alr Exist 54 CV
SN
Objects in Test: 62
Objects in Complete: 1372
Successfull 1359 SN
no MANTRA 0 SN
no MJD 15 SN
Alr Exist 60 SN
OTHER
Objects in Test: 38
Objects in Complete: 821
Successfull 810 OTHER
no MANTRA 3 OTHER
no MJD 32 OTHER
Alr Exist 14 OTHER


In [ ]:
# FILE = '../tao_data/Test/AGN/CSS100213:095543+320356/01_09FEB04_N32046_0001_cutout_1713.fits'
# hdul = fits.open(FILE)
# hdul[0].header

In [ ]:
def create_fits_NON(ID,files,path_file,classification):
    path = './data/NON/'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    filename = './data/{}/{}.fits'.format(classification,ID)
    
    if os.path.isfile(filename):
#         print("Object {} already exists!".format(ID))
        return 3
    
    df = pd.read_pickle('../tao_data/all_NON.pkl')
    CRTS_ID = np.array(df['Neg_ID'])
    if sum(CRTS_ID == ID) != 0:
        ii = (df['Neg_ID'] == CRTS_ID)
        IM_REF = np.array(df['Im_ref'][ii])[0]
        OBJ = np.array(df['Obj'][ii])[0]
        ORIGIN = np.array(df['Origin'][ii])[0]
        
        N = len(files)
        RA = np.array(df[ (CRTS_ID == ID)]['RA'])[0]
        DEC = np.array(df[ (CRTS_ID == ID)]['DEC'])[0]
        HDU_EXT = np.arange(2,N+2).astype('S4')
        MJD = np.zeros(N, dtype='S11')
        hdict = {'CRTS_ID': ID, 'N_Images': N, 'RA_(J2000)': RA, 'Dec_(J2000)':DEC, 'Img_Ref': IM_REF}        
        for i, file in enumerate(files):
            try:
                try:
                    hdul = fits.open('{}.fits'.format(file[:-12]))
                    MJD[i] = str(hdul[0].header['MJD'])
                except:
                    date_obs = str(hdul[0].header['DATE-OBS'])
                    time_obs = str(hdul[0].header['TIME-OBS'])
                    time = '{}T{}'.format(date_obs,time_obs)
                    MJD[i] = Time(time, format='isot').mjd
            except:
                return 2

        data = {'HDU_Ext': HDU_EXT, 'MJD':MJD}
        fitsio.write(filename, data, header=hdict)
            
        for file in files:
            img = np.load(file)
            fitsio.write(filename, img)
        return 0
    else:
        return 1

In [ ]:
classes =  ['NON']
for classification in classes:
    print(classification)
    sucss = 0
    alr_ext = 0
    no_MJD = 0
    no_MANTRA = 0
    
    path = '../tao_data/Test/{}/'.format(classification)
    objects = !ls $path
    print('Objects in Test: {}'.format(len(objects)))
    for objct in objects[0:2]:
        path_file = path+objct+'/*.npy'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_NON(ID,files,path_file,classification)
        if value == 0:
            sucss +=1
        elif value == 1:
            no_MANTRA += 1
        elif value == 2:
            no_MJD += 1
        else:
            alr_ext += 1


#     path = '../tao_data/{}/'.format(classification)
#     objects = !ls $path
#     print('Objects in NON: {}'.format(len(objects)))
#     for objct in objects[0:2]:
#         path_file = path+objct+'/*.npy'
#         files = !ls $path_file
#         ID = objct[objct.find('CSS'):objct.find('CSS')+23]
#         value = create_fits_NON(ID,files,path_file,classification)
#         if value == 0:
#             sucss +=1
#         elif value == 1:
#             no_MANTRA += 1
#         elif value == 2:
#             no_MJD += 1
#         else:
#             alr_ext += 1
        
    print('Successfull {} {}'.format(sucss,classification))
    print('no MANTRA {} {}'.format(no_MANTRA,classification))
    print('no MJD {} {}'.format(no_MJD,classification))
    print('Alr Exist {} {}'.format(alr_ext,classification))